In [5]:
import pandas as pd
import torch

from scripts.training import run_epoch, train_loop
#from scripts.evaluating import 
from data_simulator import PsfDataset
from models.subpix_rcnn import SubpixRCNN

import torch.nn as nn
from torchvision.ops import MultiScaleRoIAlign
from torchvision.ops import misc as misc_nn_ops
from torchvision.transforms._presets import ObjectDetection
from torchvision.models._api import register_model, Weights, WeightsEnum
from torchvision.models._meta import _COCO_CATEGORIES
from torchvision.models._utils import _ovewrite_value_param, handle_legacy_interface
from torchvision.models.resnet import resnet50, ResNet50_Weights
from torchvision.models.detection._utils import overwrite_eps
from torchvision.models.detection.backbone_utils import _resnet_fpn_extractor, _validate_trainable_layers
from torchvision.models.detection.faster_rcnn import _default_anchorgen, FasterRCNN, FastRCNNConvFCHead, RPNHead
from torchvision.models.detection.roi_heads import RoIHeads

from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection.generalized_rcnn import GeneralizedRCNN
import torch.nn.functional as F
import torch.optim as optim
from data_simulator import make_one_data


In [ ]:
# Instantiate the datasets.
num_spots_min = 5
num_spots_max = 5
sigma_mean = 2.0
sigma_std = 0.0
snr_mean = 10
snr_std = 0.2
base_noise_min = 50
base_noise_max = 150
use_gauss_noise = True
gauss_noise_std = 10
img_w, img_h = 64, 64

# Instantiate the datasets.
train_dataset = PsfDataset( 1,
                            200,
                            num_spots_min, num_spots_max,
                            sigma_mean, sigma_std,
                            snr_mean, snr_std,
                            base_noise_min, base_noise_max,
                            use_gauss_noise, gauss_noise_std,
                            img_w, img_h
                          )

valid_dataset = PsfDataset( 2,
                            40,
                            num_spots_min, num_spots_max,
                            sigma_mean, sigma_std,
                            snr_mean, snr_std,
                            base_noise_min, base_noise_max,
                            use_gauss_noise, gauss_noise_std,
                            img_w, img_h
                          )


# Print the number of samples in the training and validation datasets
print(pd.Series({
    'Training dataset size:': len(train_dataset),
    'Validation dataset size:': len(valid_dataset)}))

data_loader_params = {'batch_size':4,    'collate_fn': lambda batch: tuple(zip(*batch)),}

training_loader = torch.utils.data.DataLoader(train_dataset, **data_loader_params)
validation_loader = torch.utils.data.DataLoader(valid_dataset, **data_loader_params)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

backbone = resnet_fpn_backbone("resnet50", pretrained=True)
kwargs = {"nms_tresh": 0.1, "detections_per_img": 5}
model = SubpixRCNN(backbone, num_classes=2, device=device, **kwargs)
model.to(device)
model.name = "testmodel"
optimizer = optim.Adam(model.parameters(), lr=0.001)

import os
import datetime

def generate_checkpoint_path(model_name, project_name="Subpix_models"):
  """Generates a timestamped checkpoint path for saving model state dicts.

  Args:
    model_name: The name of the model.
    project_name: The name of the project (optional, defaults to "Subpix_models").

  Returns:
    A pathlib.Path object representing the checkpoint path.
  """
  # 1. Define the project directory within Colab's content area
  folder_path = os.path.join('/content/', project_name)

  # 2. Create the directory if it doesn't exist
  os.makedirs(folder_path, exist_ok=True)

  # 3. Generate a timestamped subdirectory
  timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
  checkpoint_dir = os.path.join(folder_path, timestamp)
  os.makedirs(checkpoint_dir, exist_ok=True)

  # 4. Construct the checkpoint path
  checkpoint_path = os.path.join(checkpoint_dir, f"{model_name}.pth")

  return checkpoint_path

checkpoint_path = generate_checkpoint_path("sp_no_noise_justboxes_model")
num_epochs = 2
checkpoint_path = "/Users/august/Desktop/bachelor"
train_loop(model, training_loader, validation_loader, optimizer, device, num_epochs, checkpoint_path)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
backbone = resnet_fpn_backbone("resnet50", pretrained=True)
kwargs = {"nms_tresh": 0.1, "detections_per_img": 5}
model = SubpixRCNN(backbone, num_classes=2, device=device, **kwargs)

/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Custom SubpixRoIHeads successfully initialized!
<generator object Module.parameters at 0x17b4b6f80>


In [4]:
print(model)

SubpixRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [6]:
img, tar = make_one_data()

In [8]:
print(tar['boxes'].dtype)

torch.float32


In [9]:
import torch
from torchvision.ops import MultiScaleRoIAlign
from torchvision.models.detection.roi_heads import RoIHeads

from models.subpix_rcnn import SubpixRoIHeads, SubpixHead

# === Import your SubpixRoIHeads and SubpixHead ===
# from your_model_file import SubpixRoIHeads, SubpixHead

# Dummy FPN features (simulate 4 scales: p2, p3, p4, p5)
features = {
    "0": torch.randn(2, 256, 200, 336),  # e.g., from 800x1333 image
    "1": torch.randn(2, 256, 100, 168),
    "2": torch.randn(2, 256, 50, 84),
    "3": torch.randn(2, 256, 25, 42)
}

# Dummy proposals for two images (each with 3 boxes)
proposals = [
    torch.tensor([[10., 15., 50., 55.], [100., 120., 150., 160.], [300., 310., 330., 350.]], dtype=torch.float32),
    torch.tensor([[20., 25., 60., 65.], [120., 140., 170., 180.], [310., 320., 340., 360.]], dtype=torch.float32)
]

# Dummy image shapes (height, width) for two images
image_shapes = [(800, 1333), (800, 1333)]

# Dummy targets with 'positions' (subpixel ground truth for each box)
targets = [
    {"positions": torch.tensor([[30., 35.], [125., 140.], [315., 330.]], dtype=torch.float32)},
    {"positions": torch.tensor([[40., 45.], [145., 160.], [325., 340.]], dtype=torch.float32)},
]

# Define dummy box_head and box_predictor (not used here, so minimal)
dummy_box_head = torch.nn.Identity()
dummy_box_predictor = torch.nn.Identity()

# Initialize SubpixRoIHeads
roi_heads = SubpixRoIHeads(
    box_roi_pool=MultiScaleRoIAlign(featmap_names=["0", "1", "2", "3"], output_size=7, sampling_ratio=2),
    box_head=dummy_box_head,
    box_predictor=dummy_box_predictor,
    fg_iou_thresh=0.5,
    bg_iou_thresh=0.5,
    batch_size_per_image=512,
    positive_fraction=0.25,
    bbox_reg_weights=None,
    score_thresh=0.05,
    nms_thresh=0.5,
    detections_per_img=100,
    subpixel_head=SubpixHead(256)
)




Custom SubpixRoIHeads successfully initialized!


In [10]:
# Put in eval or train mode depending on what you want to test
roi_heads.eval()

# Pass dummy data through forward (simulate inference)
with torch.no_grad():
    output, losses = roi_heads(features, proposals, image_shapes)

print("\n=== Inference Output ===")
for i, out in enumerate(output):
    print(f"Image {i} output keys: {list(out.keys())}")
    if 'subpixel_positions' in out:
        print(f"Subpixel positions shape: {out['subpixel_positions'].shape}")
        print(out['subpixel_positions'])

# To test training mode and loss:
roi_heads.train()
output, losses = roi_heads(features, proposals, image_shapes, targets)
print("\n=== Training Losses ===")
print(losses)

ValueError: too many values to unpack (expected 2)